In [1]:
import tensorflow as tf
import numpy
import os
import json, pickle
import pandas
from functools import partial, reduce
import importlib

import sys
sys.path.append('../libs')

import flacdb
import prepare_data
import initialize
import data_pipeline
import loss_metrics
import conv_model
import plot_batch

In [2]:
codes = json.load(open('/scr-ssd/icd/icd9.json'))

In [3]:
codes[0]

[{'code': '001-139',
  'depth': 1,
  'href': '/index.php?action=child&recordid=1',
  'descr': 'INFECTIOUS AND PARASITIC DISEASES '},
 {'code': '001-009',
  'depth': 2,
  'href': '/index.php?action=child&recordid=2',
  'descr': 'INTESTINAL INFECTIOUS DISEASES '},
 {'code': '001',
  'depth': 3,
  'href': '/index.php?action=child&recordid=3',
  'descr': 'Cholera'},
 {'code': '001.9', 'depth': 4, 'href': None, 'descr': 'Cholera, unspecified'}]

In [4]:
codes[1]

[{'code': '001-139',
  'depth': 1,
  'href': '/index.php?action=child&recordid=1',
  'descr': 'INFECTIOUS AND PARASITIC DISEASES '},
 {'code': '001-009',
  'depth': 2,
  'href': '/index.php?action=child&recordid=2',
  'descr': 'INTESTINAL INFECTIOUS DISEASES '},
 {'code': '001',
  'depth': 3,
  'href': '/index.php?action=child&recordid=3',
  'descr': 'Cholera'},
 {'code': '001.1',
  'depth': 4,
  'href': None,
  'descr': 'Due to Vibrio cholerae el tor'}]

In [5]:
codes[2]

[{'code': '001-139',
  'depth': 1,
  'href': '/index.php?action=child&recordid=1',
  'descr': 'INFECTIOUS AND PARASITIC DISEASES '},
 {'code': '001-009',
  'depth': 2,
  'href': '/index.php?action=child&recordid=2',
  'descr': 'INTESTINAL INFECTIOUS DISEASES '},
 {'code': '001',
  'depth': 3,
  'href': '/index.php?action=child&recordid=3',
  'descr': 'Cholera'},
 {'code': '001.0',
  'depth': 4,
  'href': None,
  'descr': 'Due to Vibrio cholerae'}]

In [6]:
codes[3]

[{'code': '001-139',
  'depth': 1,
  'href': '/index.php?action=child&recordid=1',
  'descr': 'INFECTIOUS AND PARASITIC DISEASES '},
 {'code': '001-009',
  'depth': 2,
  'href': '/index.php?action=child&recordid=2',
  'descr': 'INTESTINAL INFECTIOUS DISEASES '},
 {'code': '002',
  'depth': 3,
  'href': '/index.php?action=child&recordid=7',
  'descr': 'Typhoid and paratyphoid fevers'},
 {'code': '002.9',
  'depth': 4,
  'href': None,
  'descr': 'Paratyphoid fever, unspecified'}]

In [43]:
names = {j['code'] for i in codes for j in i}
names = {i for i in names if i is not None}
names = {i.replace('.', '') if '.' in i else i for i in names}
len(names)

11835

In [45]:
missing = [i for i in counts.index if i not in names]
len(missing)

2021

In [50]:
[i for i in codes if i[-1]['code'] == '285']

[[{'code': '280-289',
   'depth': 1,
   'href': '/index.php?action=child&recordid=2175',
   'descr': 'DISEASES OF THE BLOOD AND BLOOD-FORMING ORGANS '},
  {'code': '285',
   'depth': 2,
   'href': '/index.php?action=child&recordid=2228',
   'descr': 'Other and unspecified anemias'},
  {'code': '285',
   'depth': 3,
   'href': None,
   'descr': 'Other and unspecified anemias'}]]

In [48]:
[i for i in names if '285' in i]

['285',
 'E9285',
 '03285',
 'V285',
 '29285',
 '11285',
 '30285',
 'V7285',
 '5285',
 '7285',
 '36285']

In [49]:
[i for i in loss_metrics.CODE_NAMES if i not in names]

['25000', '2859', '41071', '4917']

In [51]:
codes2 = pandas.read_csv('/scr-ssd/icd/ICD9DIAG.csv')

In [58]:
len(set(counts.index))

6984

In [62]:
len(set(codes2['code']) & set(counts.index))

6316

In [63]:
len(names & set(counts.index))

4963

In [72]:
S1 = names
S2 = set(codes2['code'])
S = S1 | S2
missing2 = [i for i in counts.index if i not in S]

In [75]:
len(missing2)

282

In [80]:
sum(i not in S1 for i in counts.index)

2021

In [89]:
missing = [i for i in counts.index if i not in S1 and i[:-1] not in S1]
len(missing)

353

In [112]:
[i for i in codes if any(j['code'] is not None and '428.3' in j['code'] for j in i)]

[[{'code': '390-459',
   'depth': 1,
   'href': '/index.php?action=child&recordid=4314',
   'descr': 'DISEASES OF THE CIRCULATORY SYSTEM '},
  {'code': '420-429',
   'depth': 2,
   'href': '/index.php?action=child&recordid=4447',
   'descr': 'OTHER FORMS OF HEART DISEASE '},
  {'code': '428',
   'depth': 3,
   'href': '/index.php?action=child&recordid=4534',
   'descr': 'Heart failure'},
  {'code': '428.3',
   'depth': 4,
   'href': '/index.php?action=child&recordid=4542',
   'descr': 'Diastolic heart failure'},
  {'code': '428.33', 'depth': 5, 'href': None, 'descr': 'Acute on chronic'}],
 [{'code': '390-459',
   'depth': 1,
   'href': '/index.php?action=child&recordid=4314',
   'descr': 'DISEASES OF THE CIRCULATORY SYSTEM '},
  {'code': '420-429',
   'depth': 2,
   'href': '/index.php?action=child&recordid=4447',
   'descr': 'OTHER FORMS OF HEART DISEASE '},
  {'code': '428',
   'depth': 3,
   'href': '/index.php?action=child&recordid=4534',
   'descr': 'Heart failure'},
  {'code': '4

In [107]:
counts[:50]

,Condition,Count
4019,Unspecified essential hypertension,20703
4280,"Congestive heart failure, unspecified",13111
42731,Atrial fibrillation,12891
41401,Coronary atherosclerosis of native coronary ar...,12429
5849,"Acute kidney failure, unspecified",9119
25000,Diabetes mellitus without mention of complicat...,9058
2724,Other and unspecified hyperlipidemia,8690
51881,Acute respiratory failure,7497
5990,"Urinary tract infection, site not specified",6555
53081,Esophageal reflux,6326


In [99]:
(counts['Count'] > 2000).sum()

56

In [105]:
counts.reindex(loss_metrics.CODE_NAMES)

,Condition,Count
25000,Diabetes mellitus without mention of complicat...,9058.0
2720,Pure hypercholesterolemia,5930.0
2724,Other and unspecified hyperlipidemia,8690.0
2761,Hyposmolality and/or hyponatremia,3039.0
2762,Acidosis,4528.0
27651,Dehydration,1385.0
27652,Hypovolemia,1374.0
27800,"Obesity, unspecified",1511.0
2859,"Anemia, unspecified",5406.0
4019,Unspecified essential hypertension,20703.0


In [ ]:
counts['Count'] > 1000

In [113]:
diagnosis = pandas.read_csv(clinic_file('diagnoses_icd'))
new_names = {i.upper(): i for i in ['subject_id', 'hadm_id']}
new_names['ICD9_CODE'] = 'code'
diagnosis = diagnosis[new_names].rename(columns=new_names)

In [114]:
diagnosis

,subject_id,hadm_id,code
0,109,172335,40301
1,109,172335,486
2,109,172335,58281
3,109,172335,5855
4,109,172335,4254
...,...,...,...
651042,97503,188195,20280
651043,97503,188195,V5869
651044,97503,188195,V1279
651045,97503,188195,5275


In [18]:
len(counts)

6984

In [14]:
root = '/scr1/mimic/'
clinic_file = lambda i: root + 'clinic/{}.csv'.format(i.upper())
diagnoses_info = pandas.read_csv(clinic_file('d_icd_diagnoses'))
diagnoses_info = diagnoses_info.drop(columns='ROW_ID').set_index('ICD9_CODE')
diagnoses_info.rename(columns={'LONG_TITLE': 'Condition'}, inplace=True)
diagnoses = pandas.read_csv(clinic_file('diagnoses_icd')).drop(columns='ROW_ID')
counts = diagnoses['ICD9_CODE'].value_counts()
counts = pandas.concat([
    diagnoses_info.reindex(counts.index)[['Condition']], 
    pandas.Series(counts, name='Count')
], axis=1)
counts[:50]

,Condition,Count
4019,Unspecified essential hypertension,20703
4280,"Congestive heart failure, unspecified",13111
42731,Atrial fibrillation,12891
41401,Coronary atherosclerosis of native coronary ar...,12429
5849,"Acute kidney failure, unspecified",9119
25000,Diabetes mellitus without mention of complicat...,9058
2724,Other and unspecified hyperlipidemia,8690
51881,Acute respiratory failure,7497
5990,"Urinary tract infection, site not specified",6555
53081,Esophageal reflux,6326
